<a href="https://colab.research.google.com/github/masunomatiko/lab/blob/master/Fleiss'_kappa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
df1 = pd.read_csv('./f1440230.csv')
df2 = pd.read_csv('./f1441250.csv')
df3 = pd.read_csv('./f1442307.csv').drop(columns=['how_much_is_this_news_article_relevant_to_present_users_gold_reason',\
                                                  'how_well_does_this_result_match_the_query_gold',\
                                                  'how_well_does_this_result_match_the_query_gold_reason'\
                                                  ])
df = pd.concat([df1, df2, df3])

In [3]:
df1.columns

Index(['_unit_id', '_created_at', '_golden', '_id', '_missed', '_started_at',
       '_tainted', '_channel', '_trust', '_worker_id', '_country', '_region',
       '_city', '_ip',
       'how_much_is_this_news_article_relevant_to_present_users', 'doc_id',
       'how_much_is_this_news_article_relevant_to_present_users_gold', 'query',
       'text', 'title', 'year'],
      dtype='object')

In [4]:
df3.columns

Index(['_unit_id', '_created_at', '_golden', '_id', '_missed', '_started_at',
       '_tainted', '_channel', '_trust', '_worker_id', '_country', '_region',
       '_city', '_ip',
       'how_much_is_this_news_article_relevant_to_present_users', 'doc_id',
       'how_much_is_this_news_article_relevant_to_present_users_gold', 'query',
       'text', 'title', 'year'],
      dtype='object')

In [5]:
df1.shape, df2.shape, df3.shape

((4738, 21), (1222, 21), (4103, 21))

In [6]:
df.head()

,_unit_id,_created_at,_golden,_id,_missed,_started_at,_tainted,_channel,_trust,_worker_id,_country,_region,_city,_ip,how_much_is_this_news_article_relevant_to_present_users,doc_id,how_much_is_this_news_article_relevant_to_present_users_gold,query,text,title,year
0,2493064550,10/5/2019 02:17:52,False,5216643884,NaN,10/5/2019 02:17:32,False,clixsense,0.8125,45447404,USA,NY,New York,104.131.66.8,4_strongly_relevant,851148,NaN,acquired immune deficiency syndrome (aids),The Centers for Disease Control and Prevention...,Growing Fast As a Factor In AIDS: Drugs,1996
1,2493064550,10/5/2019 02:18:38,False,5216644712,NaN,10/5/2019 02:17:23,False,feca,0.6250,45438493,USA,AZ,Phoenix,172.241.239.97,4_strongly_relevant,851148,NaN,acquired immune deficiency syndrome (aids),The Centers for Disease Control and Prevention...,Growing Fast As a Factor In AIDS: Drugs,1996
2,2493064550,10/5/2019 02:19:29,False,5216645574,NaN,10/5/2019 02:17:23,False,feca,0.7778,45418638,VEN,07,Valencia,200.84.151.88,3_moderately_relevant,851148,NaN,acquired immune deficiency syndrome (aids),The Centers for Disease Control and Prevention...,Growing Fast As a Factor In AIDS: Drugs,1996
3,2493064551,10/5/2019 02:14:57,False,5216640885,NaN,10/5/2019 02:14:35,False,feca,0.7500,45332116,USA,NaN,NaN,38.132.116.171,3_moderately_relevant,1028898,NaN,acquired immune deficiency syndrome (aids),This helps answer a major question that has st...,"Immune System Can Revive After AIDS, Studies S...",1998
4,2493064551,10/5/2019 02:15:08,False,5216641048,NaN,10/5/2019 02:14:37,False,feca,0.7692,45438681,USA,CA,San Francisco,104.236.195.72,3_moderately_relevant,1028898,NaN,acquired immune deficiency syndrome (aids),This helps answer a major question that has st...,"Immune System Can Revive After AIDS, Studies S...",1998


In [0]:
labels = ['1_not_relevant',
 '2_weakly_relevant',
 '3_moderately_relevant',
 '4_strongly_relevant']

In [0]:
tmp = pd.pivot_table(df.query('how_much_is_this_news_article_relevant_to_present_users_gold not in {}'.format(labels)), values='_worker_id', index='doc_id', columns='how_much_is_this_news_article_relevant_to_present_users', aggfunc='count').fillna(0)

In [0]:
tmp['sum'] = tmp.sum(axis=1)

In [0]:
tmp2 = tmp[tmp['sum']==3].drop(columns=['sum'])

In [11]:
tmp2.values

array([[0., 0., 3., 0.],
       [0., 0., 1., 2.],
       [1., 1., 0., 1.],
       ...,
       [0., 0., 2., 1.],
       [0., 1., 2., 0.],
       [0., 2., 1., 0.]])

In [0]:
def calc_fleiss_kappa(eval_data):
    """
    calculate fleiss's kappa score for input evaluation data
    Reference: https://en.wikipedia.org/wiki/Fleiss%27_kappa
    
    :params numpy.ndarray eval_data: num_of_targets * num_of_categories
    :return: p_bar, pe_bar, fleiss's kappa score
    :rtype: numpy.float, numpy.float, numpy.float|float
    """
    N, k = eval_data.shape
    n = np.sum(eval_data[0])
    
    # Verification of input parameters
    if n < 2:
        assert n < 2, 'The number of workers must be two or more.'
    if not np.all(np.sum(eval_data, axis=1)==n):
        
        assert np.all(np.sum(eval_data, axis=1)==n), 'The same number of workers must be assined to any subjects.'
    
    p_bar = (np.sum(np.power(eval_data, 2)) - (N * n)) / (N * n * (n - 1))
    pe_bar = np.sum(np.power(np.sum(eval_data, axis=0) / (N * n), 2))
    
    try:
        kappa = (p_bar - pe_bar) / (1 - pe_bar)
        return p_bar, pe_bar, kappa
    except ZeroDivisionError as e:
        # All worker's evaluations are probably equal.
        return p_bar, pe_bar, 1.0

In [13]:
p_bar, pe_bar, kappa_score = calc_fleiss_kappa(np.array(tmp2.values))
print('p_bar = {:.3f}, pe_bar = {:.3f}, kappa = {:.3f}'.format(p_bar, pe_bar, kappa_score))

p_bar = 0.329, pe_bar = 0.303, kappa = 0.036
